# Updating the data pipeline

This Jupyter Notebook will focus on taking the example data pipeline and applying better coding practices.  The actual code itself and the way the pipeline process information will largely be the same.  

This will include, but not be limited to:

* Adding upper case text for constants.

* Reorganising functions to clearly lay out ETL principles.

* Generally make the program more legible.

Original code found here: https://github.com/thewong-andonly/example-data-pipeline)

# Import tools

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
from requests import get
import lxml
import time
import csv
import os
import datetime

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## Extract - scrape_data()

This is the function doing most of the hard work and forms the "Extract" part of the ETL pipeline.  It uses Selenium to navigate through web pages and a mix of Selenium and BeautifulSoup to parse the data.

In [ ]:
def scrape_data():
    """
    Combination of table 1 and table 2.  Does the following:
    
    1. Parses the hyperlink from the dynamic main page to give a static page.
    
    2. Parses static page for relevant information.
    
    3. Parses main page for relevant information.
    
    `OUTPUT = True` prints all of the scraped details.  
    `OUTPUT = False` turns off verbose mode.
    
    write:
    `WRITE = True` calls `process _data()` function and saves the file to the current working directory to a csv.
    `WRITE = False` does not save the file.
    """
    i = 0
    table = DRIVER.find_elements_by_xpath("//a[@class='pointer']")
    for link in table:
        elements = DRIVER.find_elements_by_tag_name('td')
        
        # Counts the number of columns and number of rows
        columns = DRIVER.find_elements_by_tag_name('tr')
        number_of_columns = (len(columns[0].text.split()))
        number_of_rows = len(elements[::number_of_columns])
        
        # Filter out language hyperlinks
        hyperlink = link.get_attribute("href")
        if 'analysis' in hyperlink:
           
            # Scraping hyperlink data:
            
            url = get(hyperlink)
            soup = BeautifulSoup(url.text, 'html.parser')
            table = soup.findAll("li", {"class":"list-group-item"})

            # Profits
            pr = soup.findAll("div", {"class": "number"})
            pro = pr[2].text.strip()
            profit = pro.encode('ascii', errors='ignore')

            # Balance
            ba = soup.find("li", class_="list-group-item")
            bal = ba.text.strip().split()
            balance = bal[1].encode('ascii', errors='ignore')

            # Equity
            eq = table[1].text.split()
            equity = eq[2].encode('ascii', errors='ignore')

            # Deposits
            de = table[3].text.split()
            deposits = de[1].encode('ascii', errors='ignore')

            # Withdrawals
            wi=table[4].text.strip().split()
            withdrawals=wi[1].encode('ascii', errors='ignore')

            # Trades
            tr = table[5].text.split()
            trades = tr[1]

            # Won
            wo = table[7].text.strip().split()
            won = wo[1].encode('ascii', errors='ignore')

            # Average trade time
            avg_trade_time = table[8].text.strip().split()
            att = ' '.join(avg_trade_time[3:5])
            
            if len(att) == 1:
                att_hours = att
            if len(att) > 4:
                h = att.split('h')
                hours = int(h[0])
                m = h[1].split('m')
                mins = int(m[0])
                att_hours = (hours+mins/60)
            if len(att) <= 4:
                if 'm' in att:
                    m = att.split('m')
                    mins = int(m[0])
                    att_hours = mins/60
                if 'd' in att:
                    d = att.split('d')
                    days= int(d[0])
                    att_hours = days*24
                
            # Profit Factor
            pf = table[10].text.strip().split()
            profit_factor = pf[2].encode('ascii', errors='ignore')

            # Daily
            da = table[11].text.strip().split()
            daily = da[1].encode('ascii', errors='ignore')

            # Monthly
            mo = table[12].text.strip().split()
            monthly = mo[1].encode('ascii', errors='ignore')

            # Trades per month
            tpm_ = table[13].text.strip().split()
            tpm = tpm_[3].encode('ascii', errors='ignore')

            # Expectancy
            ex = table[14].text.strip().split()
            expectancy = ex[4].encode('ascii', errors='ignore')

            t2 = soup.find("div", {"class":"caption-helper font-blue-sharp bold master-description-container"})
            t2_parsed = t2.text.split(', ')

            # Rank
            if '#' in t2_parsed[0]:
                r = t2_parsed[0].split('#')
                rank = r[1]
            if '#' not in t2_parsed[0]:
                rank = '-'

            # Platform
            platform = t2_parsed[3]

            # Ratio
            ratio = t2_parsed[4].split(':')
            ratio1 = ratio[0]
            ratio2 = ratio[1]

            # Platform 2
            platform2 = t2_parsed[5]

            # Info
            info_table = soup.find("p")
            i_ = info_table.text.strip()
            info = i_.encode('ascii', errors='ignore')
            
            # Scraping search page data:
            rank = elements[i].text
            name = elements[i+1].text
            gain = elements[i+2].text
            pips = elements[i+3].text
            dd = elements[i+4].text
            trades = elements[i+5].text
            price = elements[i+8].text
            age = elements[i+9].text
            added = elements[i+10].text
            i += number_of_columns
    
            # Optional output
            if OUTPUT == True:
                print(f'Data for {name}.\nHyperlink: {hyperlink}\n')
                #table 1/main page
                print(f'Rank: {rank}')
                print(f'Name: {name}') 
                print(f'Gain: {gain}') 
                print(f'Pips: {pips}') 
                print(f'DD: {dd}')
                print(f'Trades: {trades}')
                print(f'Price: {price} ')
                print(f'Age: {age}')
                print(f'Added: {added}')

                #hyperlink data
                print(f'Rank: {rank}')
                print(f'Platform: {platform}')
                print(f'Ratio: {ratio1}:{ratio2}')
                print(f'Platform 2: {platform2}')
                print(f"Profit: {profit}")
                print(f"Balance: {balance}")
                print(f"Equity: {equity}")
                print(f"Deposits: {deposits}")
                print(f"Withdrawals: {withdrawals}")
                print(f"Trades: {trades}")
                print(f"Won %: {won}")
                print(f"Average trade time: {att}")
                print(f"Average trade time (hours): {att_hours}")
                print(f"Profit factor: {profit_factor}")
                print(f'Daily: {daily}')
                print(f'Monthly: {monthly}')
                print(f'Trades per month: {tpm}')
                print(f'Expectancy: {expectancy}')
                print(f'Info:{info}\n')

            # Transform - write to CSV file
            if WRITE == True:
                # All columns for CSV
                all_signals_file_columns = [
                    'Rank', 
                    'Name', 
                    'Hyperlink', 
                    'Gain', 
                    'Pips', 
                    'DD', 
                    'Trades', 
                    'Price', 
                    'Age', 
                    'Added', 
                    'Platform', 
                    'Ratio1', 
                    'Ratio2', 
                    'Platform 2', 
                    'Profit', 
                    'Balance', 
                    'Equity', 
                    'Deposits', 
                    'Withdrawals', 
                    'Trades', 
                    'Won', 
                    'Average Trade Time', 
                    'Average Trade time (hours)', 
                    'Profit Factor',
                    'Daily', 
                    'Monthly', 
                    'Trades per month', 
                    'Expectancy']
                
                all_signals_file_data = [
                    [rank, 
                    name, 
                    hyperlink, 
                    gain, 
                    pips, 
                    dd, 
                    trades, 
                    price, 
                    age, 
                    added, 
                    platform, 
                    ratio1, 
                    ratio2, 
                    platform2, 
                    profit, 
                    balance, 
                    equity, 
                    deposits, 
                    withdrawals, 
                    trades, 
                    won, 
                    att, 
                    att_hours, 
                    profit_factor, 
                    daily, 
                    monthly, 
                    tpm, 
                    expectancy]
                    ]
                signals_info_file_data = [[name, info]]
               
                signals_info_file_columns = ['Name', 'Info']

                # Transform
                process_data('all-signals-no-info', all_signals_file_columns, all_signals_file_data, clean=True)
                process_data('all-signals-info-only', signals_info_file_columns, signals_info_file_data, clean=False)

## Transform - process_data()
This is used in the "Transform" section of the ETL pipeline by converting the extracted data into a .csv format and cleaning it appropriately using a RegEx function and the Pandas library.

In [ ]:
def process_data(target_filename, column_headers=[], data_fields=[], clean=None):
    """
    Writes data to CSV and returns CSV as raw data
    
    `column_headers` inputted as as list
    `data_fields` inputted as a list of a list
    
    clean:
    `clean = True` passes the target file to the clean_data function which produces columns with digits and punctuation only.
    `clean = False` does not pass the target file.
    """
    # Formatting file name
    date = datetime.datetime.now().strftime('%d-%m-%Y')
    file_name = date+'-'+target_filename+'.csv'
    
    # Creating data file
    raw_data_file_exists = os.path.isfile(file_name)
    with open(file_name, 'a') as file:
        if not raw_data_file_exists:
                # Add header once
                fields = column_headers
                writer = csv.DictWriter(file, fieldnames=fields)
                writer.writeheader()
        writer = csv.writer(file)
        for d in data_fields:
            writer.writerow(d)
    
    # Creating list of files for cleaning
    if clean == True:
        global data_to_clean
        data_to_clean = []
        data_to_clean.append(file_name)
        return data_to_clean

## Transform - clean_data()

Runs a RegEx to delete unwanted characters from the signal data.

RegEx here: `str.replace(r'[a-zA-Z%$\'\+]+', '')`

In [ ]:
def clean_data(target_filename):
    df = pd.read_csv(target_filename)
    
    # Drop unnamed columns
    for c in df.columns:
        if 'Unnamed' in c:
            df = df.drop(c, axis=1)
    
    # Strip away all characters which aren't numbers or punctuation for certain columns
    num_only_columns = [
    'Gain',
    'DD',
    'Price',
    'Profit', 
    'Balance', 
    'Equity', 
    'Deposits', 
    'Withdrawals', 
    'Won', 
    'Profit Factor', 
    'Daily', 
    'Monthly', 
    'Trades per month', 
    'Expectancy'
    ]
    
    for c in num_only_columns:
            df[c] = df[c].str.replace(r'[a-zA-Z%$\'\+]+', '')
    
    # Rename certain columns
    df = df.rename({'Gain': 'Gain (%)',
                    'DD': 'DD (%)',
                    'Price': 'Price ($)',
                    'Trades.1': 'Trades',
                    'Won': 'Won %',
                    'Daily': 'Daily (%)',
                    'Monthly': 'Monthly (%)'})
    
    # Save formatted data
    df.to_csv('cleaned-'+target_filename)

## Extract, Transform - Main program



In [ ]:
%%time

# Initialise Webdriver
URL = # target website
DRIVER = webdriver.Chrome()
DRIVER.get(URL)
#DRIVER.maximize_window()

# Program params:

# OUTPUT = True, provides print statements of data.
# OUTPUT = False, provides no output.
OUTPUT = True

# WRITE = True, saves the data to a csv.
# WRITE = False, doesn't save a file (for testing purposes).
WRITE = True

# CLEAN = True, uses a RegEx to delete unwanted characters.  Recommended to be True for signal data.
# CLEAN = False, does not use the RegEx. Recommended to be true for info only.
CLEAN = True

# LOAD = True, opens up target file in Pandas.
# LOAD = False, doesn't open up file.
LOAD = True

# Define number of pages
page_count = DRIVER.find_element_by_class_name('pagination-panel-total')
pages = int(page_count.text)
print(f'Scraping {URL} ...')

# Set for when loop ends
i = 1
for num in range(pages+1):
    if i > 1:
        time.sleep(2)
    print(f'Scraping page {i} of {pages} ...\n')
    
    # Extract & Transform (webscraping and saving to CSV)
    scrape_data()

    print(f'Page {i} scraped.\n')
    
    # Determine if loop continues
    i += 1
    if i == pages + 1:
        DRIVER.close()
        print('All pages scraped!')
        break
    if i != pages:
        time.sleep(1)
        WebDriverWait(DRIVER, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@class='btn btn-sm default next ']")))
        button = DRIVER.find_element_by_xpath("//a[@class='btn btn-sm default next ']")
        button.click()
        print(f'Loading page {i}...')
        print(f'Page {i} loaded.\n')

# Transform (data cleaning)
if CLEAN == True:
    print('\nCleaning data...\n')
    for file in data_to_clean:
        clean_data(file)
    print(f'{file} cleaned!')

# Load   
if LOAD == True:
    df = pd.read_csv(file)
    df